In [19]:
function get_tdl(fs, SCS, PRBS, DS, chan)
    # Check the type of channel
    if chan == "tdlc"
        coef_start = 1
        # TDL-C channel coefficients
        tdl = [1 0 -4.4;
               2 0.2099 -1.2;
               # ... (additional coefficients)
               24 8.6523 -22.8]
    elseif chan == "tdld"
        coef_start = 2
        # TDL-D channel coefficients
        tdl = [1 0 -.00147;
               2 0.035 -18.8;
               # ... (additional coefficients)
               13 12.525 -27.7]
    else
        error("unknown channel $chan")
    end

    # Compute the normalization factor for channel coefficients
    tdl_norm = sum(10 .^(0.1 * tdl[coef_start:end, 3]))

    # Generate frequency grid for the channel
    PRB = 1:12
    fgrid = []
    for PRBs in PRBS
        append!(fgrid, SCS * ((PRBs * 12) .+ PRB))
    end

    # compute channel impulse response
    Htdl = [
        exp.(-im * 2 * π * f * tdl[coef_start:end, 2] * DS) .* sqrt.(10 .^(0.1 * tdl[coef_start:end, 3]))
        for f in fgrid
    ]

    # Generate random complex gains for each channel tap
    gv = (√(0.5) * randn(length(tdl[coef_start:end, 2])) + im * √(0.5) * randn(length(tdl[coef_start:end, 2])))'

    # Combine channel response and gains
    H = Htdl * gv / sqrt(tdl_norm)

    # Add mean for Ricean channel
    if coef_start == 2
        H += exp(-im * 2 * π * f * tdl[1, 2] * DS) .* √(10 .^(0.1 * tdl[1, 3]))' / √(tdl_norm)
    end

    return H
end

get_tdl (generic function with 1 method)

In [23]:
# Example 1: TDL-C Channel
fs = 122.88e6
SCS = 30e3
PRBS = 0:105
DS = 300e-9
chan = "tdlc"

H_tdlc = get_tdl(fs, SCS, PRBS, DS, chan)
# println("TDL-C Channel Impulse Response: ", H_tdlc)



1272×3 Matrix{Vector{ComplexF64}}:
 [-0.343549+0.0622789im, -0.495475+0.0959082im, -0.0329385+0.0260212im]  …  [0.155567+1.16295im, 0.244799+1.67818im, 0.082221+0.114622im]
 [-0.343549+0.0622789im, -0.494302+0.101782im, -0.0168443+0.0384489im]      [0.155567+1.16295im, 0.264701+1.67516im, 0.126445+0.0625312im]
 [-0.343549+0.0622789im, -0.493059+0.107642im, 0.00320247+0.0418545im]      [0.155567+1.16295im, 0.284565+1.6719im, 0.140999-0.00423296im]
 [-0.343549+0.0622789im, -0.491747+0.113487im, 0.0224978+0.0354387im]       [0.155567+1.16295im, 0.304389+1.6684im, 0.122466-0.0700039im]
 [-0.343549+0.0622789im, -0.490365+0.119316im, 0.0365139+0.0207071im]       [0.155567+1.16295im, 0.32417+1.66467im, 0.0751969-0.119348im]
 [-0.343549+0.0622789im, -0.488914+0.125127im, 0.041962+0.00111656im]    …  [0.155567+1.16295im, 0.343906+1.66071im, 0.0102822-0.140687im]
 [-0.343549+0.0622789im, -0.487395+0.130922im, 0.0375635-0.018736im]        [0.155567+1.16295im, 0.363593+1.65651im, -0.0570452-0.1290

In [24]:
# Example 2: TDL-D Channel
chan = "tdld"

H_tdld = get_tdl(fs, SCS, PRBS, DS, chan)
println("TDL-D Channel Impulse Response: ", H_tdld)


LoadError: UndefVarError: `f` not defined